# Working with Sets in `pctheory`

To make a pitch-class set, we simply use the Python `set` data structure.

In [2]:
from pctheory import pitch, pcset, pcseg, transformations

# A tetrachordal set
set1 = {pitch.PitchClass(1), pitch.PitchClass(5), pitch.PitchClass(7), pitch.PitchClass(4)}

# The same tetrachordal set, but with less typing
set1 = pcset.make_pcset12(1, 5, 7, 4)

print(set1)

{4, 1, 5, 7}


Now we can do some work with a set. Let's start by getting some useful information, such as the set-class prime form and interval-class vector:

In [3]:
sc1 = pcset.SetClass(set1)
print("Prime form name:", sc1.name_prime)
print("Forte name:", sc1.name_forte)
print("Interval-class vector:", sc1.ic_vector)

Prime form name: [0236]
Forte name: 4-12
Interval-class vector: [1, 1, 2, 1, 0, 1]


It's often useful to know the abstract complement set-class:

In [4]:
complement_sc = sc1.get_abstract_complement()
print(complement_sc.name_prime)

[01345679]


What if we want to use this set-class in a piece of music, and we specifically want a member set of the class that has the pitch-classes `5` and `6` in it?

The approach we'll use is to make a pitch-class set $B$ with `5` and `6` in it, generate all transformations of the set-class we're searching, then look for transformed sets that contain $B$ as a subset.

The code below is a bit more complex. It basically generates all transpositions and inversions of the prime-form set, and then filters this collection so we are only looking at the sets that have $B$ as a subset.

In [5]:
b = pcset.make_pcset12(5, 6)
corpus = pcset.get_corpus(sc1.pcset)
member_sets = filter(lambda x: b.issubset(x), corpus)
for member in member_sets:
    print(member)

frozenset({6, 9, 5, 3})
frozenset({6, 5, 8, 2})


Now we'd like to know what transformation was applied to the prime form of the set-class to get these member sets. Here's how we do that:

In [6]:
tx = [transformations.find_utos(sc1.pcset, set(x)) for x in filter(lambda x: b.issubset(x), corpus)]
print(tx)

[{T3}, {T8M11}]


We got a list of *sets* of transformations. This is because sometimes there are multiple transformations that generate the same pitch-class set. That would happen if the *degree of symmetry* of the set-class is greater than 1 (which means that the set-class has 12 or fewer unique sets in it, rather than the normal 24).

Next, we'd like to look at all of the abstract set-classes that can be found in this set-class.

In [7]:
abstract_subset_classes = sc1.get_abstract_subset_classes()
print(abstract_subset_classes)

{(2-1)[01], (2-2)[02], (2-6)[06], (3-2)[013], (1-1)[0], (3-3)[014], (2-3)[03], (3-10)[036], (2-4)[04], (0-1)[], (4-12)[0236], (3-8)[026]}


Set-classes print using their "Morris" name - a combination of the Forte name and the prime form name.

What we just did was related to Allen Forte's concept of "set complexes." You can generate these as well if you want:

In [8]:
from pctheory import set_complex

# Get Forte's "KH" set-complex about nexus set sc1:
kh = set_complex.get_kh12(sc1)
print(kh)

[(0-1)[], (1-1)[0], (2-1)[01], (2-2)[02], (2-3)[03], (2-4)[04], (2-6)[06], (3-2)[013], (3-3)[014], (3-8)[026], (3-10)[036], (4-12)[0236], (5-4)[01236], (5-10)[01346], (5-16)[01347], (5-31)[01369], (5-Z18)[01457], (5-8)[02346], (5-28)[02368], (5-26)[02458], (6-2)[012346], (6-Z36)[012347], (6-5)[012367], (6-Z42)[012369], (6-15)[012458], (6-Z10)[013457], (6-27)[013469], (6-Z49)[013479], (6-34)[013579], (6-30)[013679], (6-31)[014579], (6-21)[023468], (6-Z45)[023469], (6-Z3)[012356], (6-Z13)[013467], (6-Z39)[023458], (6-Z28)[013569], (6-Z23)[023568], (7-4)[0123467], (7-10)[0123469], (7-16)[0123569], (7-31)[0134679], (7-Z18)[0145679], (7-8)[0234568], (7-28)[0135679], (7-26)[0134579], (8-12)[01345679], (9-2)[012345679], (9-3)[012345689], (9-8)[01234678A], (9-10)[01234679A], (10-1)[0123456789], (10-2)[012345678A], (10-3)[012345679A], (10-4)[012345689A], (10-6)[012346789A], (11-1)[0123456789A], (12-1)[0123456789AB]]


One other interesting feature is the ability to partition a set-class. This is a concept used in twelve-tone theory. For example, say we have a hexachord that we'd like to partition in two. You can list all possible set-class partitions this way:

In [9]:
sc2 = pcset.SetClass(pcset.make_pcset12(5, 2, 3, 6, 9, 11))
partitions = sc2.get_partition2_subset_classes()
print(partitions)

{((3-2)[013], (3-7)[025]), ((0-1)[], (6-Z49)[013479]), ((2-2)[02], (4-3)[0134]), ((3-11)[037], (3-11)[037]), ((3-5)[016], (3-5)[016]), ((3-10)[036], (3-10)[036]), ((2-3)[03], (4-27)[0258]), ((2-4)[04], (4-25)[0268]), ((2-4)[04], (4-17)[0347]), ((3-3)[014], (3-8)[026]), ((2-6)[06], (4-18)[0147]), ((3-3)[014], (3-3)[014]), ((3-7)[025], (3-2)[013]), ((2-5)[05], (4-Z29)[0137]), ((1-1)[0], (5-16)[01347]), ((1-1)[0], (5-32)[01469]), ((1-1)[0], (5-28)[02368]), ((2-1)[01], (4-Z15)[0146]), ((2-3)[03], (4-12)[0236]), ((2-2)[02], (4-26)[0358]), ((3-11)[037], (3-8)[026])}
